In [1]:
library(boot)
library(glmnet)

Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-16



In [2]:
set.seed(2019)

In [3]:
train <- read.csv("trainC.csv")
test <- read.csv("testC.csv")
train <- subset(train, select = -c(sessionDate, trialNum, timeSinceKetamine))
test <- subset(test, select = -c(sessionDate, trialNum, timeSinceKetamine))

#TRAIN 1: ALL COVARIATES PLUS INTERACTION TERMS
train1 <- read.csv("trainC.csv")
test1 <- read.csv("testC.csv")
train1 <- subset(train1, select = -c(sessionDate, trialNum, timeSinceKetamine))
test1 <- subset(test1, select = -c(sessionDate, trialNum, timeSinceKetamine))

#TRAIN 2: ALL COVARIATES NO INTERACTION TERMS
train2 <- subset(train1, select = c(totalCellNum,gender,genotype,weight_g,ketamine_day,
                                    correlationScore,lickAccuracy,lickNumber,avgFR,
                                    avgSingleCellVariance,varianceFR,avgTrialSpeed,
                                    varianceSpeed,medianCellDepth,ketBool))
test2 <- subset(test1, select = c(totalCellNum,gender,genotype,weight_g,ketamine_day,
                                    correlationScore,lickAccuracy,lickNumber,avgFR,
                                    avgSingleCellVariance,varianceFR,avgTrialSpeed,
                                    varianceSpeed,medianCellDepth,ketBool))

In [4]:
# First, let's do a 50% split on the training data to determine the best lambda
n = length(train[,1])
n50 = round(n/2)
train50A = train[1:n50,]
train50B = train[(n50+1):n,]

## Basic Logistic Regression Model with Interaction Terms

### Estimate test error

In [5]:
k = 10
n = length(train1[,1])
fsize = round(n/k)
rmse = rep(0,k)
zoloss = rep(0,k)
for (i in 1:(k-1)){
    # Get train and validation sets
    df_train <- train1[-(((i-1)*fsize+1):(i*fsize)),]
    df_val <- train1[((i-1)*fsize+1):(i*fsize),]
    # Fit model on training and make predictions on validation
    model_cv <- glm(ketBool ~ . + animalName:correlationScore
                    + animalName:lickAccuracy
                    + animalName:lickNumber
                    + animalName:avgFR
                    + animalName:avgSingleCellVariance
                    + animalName:varianceFR
                    + animalName:avgTrialSpeed
                    + animalName:varianceSpeed, data=df_train, family='binomial')
    lr_pred_lo <- predict(model_cv,df_val) # lo : log odds
    num_val = length(df_val$ketBool)
    lr_pred = rep(0,num_val)
    actual = rep(0,num_val)
    for (j in 1:num_val){
        if (lr_pred_lo[j]>0){
            lr_pred[j]=1
        }
    actual[j] = df_val$ketBool[j]
    }
    # Compute 0-1 loss for each observation
    lr_loss = abs(lr_pred-actual) # loss is 0 if NB_pred=actual, 1 otherwise
    # Compute mean 0-1 loss on the val set
    zoloss[i] = mean(lr_loss)
}
df_train <- train1[-(((k-1)*fsize+1):n),]
df_val <- train1[((k-1)*fsize+1):n,]
# Fit model on training and make predictions on validation
model_cv <- glm(ketBool ~ . + animalName:correlationScore
                    + animalName:lickAccuracy
                    + animalName:lickNumber
                    + animalName:avgFR
                    + animalName:avgSingleCellVariance
                    + animalName:varianceFR
                    + animalName:avgTrialSpeed
                    + animalName:varianceSpeed, data=df_train, family='binomial')
lr_pred_lo <- predict(model_cv,df_val) # lo : log odds
num_val = length(df_val$ketBool)
lr_pred = rep(0,num_val)
actual = rep(0,num_val)
for (j in 1:num_val){
    if (lr_pred_lo[j]>0){
        lr_pred[j]=1
    }
    actual[j] = df_val$ketBool[j]
}
lr_loss = abs(lr_pred-actual)
zoloss[k] = mean(lr_loss)
test_error_est = mean(zoloss)

cat("=====================================================================\n")
cat("Logistic Regression Model with Interaction Terms\n\n")
cat("Zero-One Loss (10-fold Cross-Validation Average):",test_error_est,"\n")
cat("Accuracy (10-fold Cross-Validation Average):",1-test_error_est,"\n")
cat("=====================================================================\n")

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
“prediction from a rank-deficient fit may be misleading”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message in predict.lm(object, newdata, s

Logistic Regression Model with Interaction Terms

Zero-One Loss (10-fold Cross-Validation Average): 0.05053967 
Accuracy (10-fold Cross-Validation Average): 0.9494603 


In [6]:
summary(model_cv)


Call:
glm(formula = ketBool ~ . + animalName:correlationScore + animalName:lickAccuracy + 
    animalName:lickNumber + animalName:avgFR + animalName:avgSingleCellVariance + 
    animalName:varianceFR + animalName:avgTrialSpeed + animalName:varianceSpeed, 
    family = "binomial", data = df_train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.5384  -0.1027   0.0000   0.0712   5.5159  

Coefficients: (18 not defined because of singularities)
                                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)                            -27.2669     6.0252  -4.526 6.03e-06 ***
animalNameG2                             0.4197     3.3163   0.127 0.899281    
animalNameG3                            -0.9364     3.5589  -0.263 0.792468    
animalNameG4                             1.4332     3.8333   0.374 0.708496    
animalNameG5                            -8.9931     4.1376  -2.173 0.029743 *  
animalNameHCN1                         -14.0175   

## Basic Logistic Regression without Interaction Terms

In [7]:
k = 10
n = length(train2[,1])
fsize = round(n/k)
rmse = rep(0,k)
zoloss = rep(0,k)
for (i in 1:(k-1)){
    # Get train and validation sets
    df_train <- train2[-(((i-1)*fsize+1):(i*fsize)),]
    df_val <- train2[((i-1)*fsize+1):(i*fsize),]
    # Fit model on training and make predictions on validation
    model_cv <- glm(ketBool ~ ., data=df_train, family='binomial')
    lr_pred_lo <- predict(model_cv,df_val) # lo : log odds
    num_val = length(df_val$ketBool)
    lr_pred = rep(0,num_val)
    actual = rep(0,num_val)
    for (j in 1:num_val){
        if (lr_pred_lo[j]>0){
            lr_pred[j]=1
        }
    actual[j] = df_val$ketBool[j]
    }
    # Compute 0-1 loss for each observation
    lr_loss = abs(lr_pred-actual) # loss is 0 if NB_pred=actual, 1 otherwise
    # Compute mean 0-1 loss on the val set
    zoloss[i] = mean(lr_loss)
}
df_train <- train2[-(((k-1)*fsize+1):n),]
df_val <- train2[((k-1)*fsize+1):n,]
# Fit model on training and make predictions on validation
model_cv <- glm(ketBool ~ ., data=df_train, family='binomial')
lr_pred_lo <- predict(model_cv,df_val) # lo : log odds
num_val = length(df_val$ketBool)
lr_pred = rep(0,num_val)
actual = rep(0,num_val)
for (j in 1:num_val){
    if (lr_pred_lo[j]>0){
        lr_pred[j]=1
    }
    actual[j] = df_val$ketBool[j]
}
lr_loss = abs(lr_pred-actual)
zoloss[k] = mean(lr_loss)
test_error_est = mean(zoloss)

cat("=====================================================================\n")
cat("Logistic Regression Model without Interaction Terms\n\n")
cat("Zero-One Loss (10-fold Cross-Validation Average):",test_error_est,"\n")
cat("Accuracy (10-fold Cross-Validation Average):",1-test_error_est,"\n")
cat("=====================================================================\n")

Logistic Regression Model without Interaction Terms

Zero-One Loss (10-fold Cross-Validation Average): 0.1413709 
Accuracy (10-fold Cross-Validation Average): 0.8586291 


# GLMNET

In [27]:
# First, let's do a 50% split on the training data to determine the best lambda
n = length(train[,1])
n50 = round(n/2)
train50A = train[1:n50,]
train50B = train[(n50+1):n,]

xA = model.matrix(ketBool ~ . + animalName:correlationScore
                    + animalName:lickAccuracy
                    + animalName:lickNumber
                    + animalName:avgFR
                    + animalName:avgSingleCellVariance
                    + animalName:varianceFR
                    + animalName:avgTrialSpeed
                    + animalName:varianceSpeed-1, data = train50A)
yA = train50A$ketBool

xB = model.matrix(ketBool ~ . + animalName:correlationScore
                    + animalName:lickAccuracy
                    + animalName:lickNumber
                    + animalName:avgFR
                    + animalName:avgSingleCellVariance
                    + animalName:varianceFR
                    + animalName:avgTrialSpeed
                    + animalName:varianceSpeed-1, data = train50B)
yB = train50B$ketBool

## Lasso

In [28]:
# Select regularization parameter over trainA (50% of training data)
model_lasso <- cv.glmnet(xA, yA, family='binomial',alpha=1)
lambda_min_lasso = model_lasso$lambda.min
lambda_1se_lasso = model_lasso$lambda.1se
cat("lambda_min_lasso = ",lambda_min_lasso,"\n")
cat("lambda_1se_lasso = ",lambda_1se_lasso,"\n")

lambda_min_lasso =  0.001487149 
lambda_1se_lasso =  0.004138081 


## lambda.min

In [29]:
k = 10
n = length(train50B[,1])
fsize = round(n/k)
rmse = rep(0,k)
zoloss = rep(0,k)
for (i in 1:(k-1)){
    # Get train and validation sets
    xB_train = xB[-(((i-1)*fsize+1):(i*fsize)),]
    yB_train = yB[-(((i-1)*fsize+1):(i*fsize))]
    xB_val = xB[((i-1)*fsize+1):(i*fsize),]
    yB_val = yB[((i-1)*fsize+1):(i*fsize)]
    # Fit model on training and make predictions on validation
    model_cv <- glmnet(xB_train, yB_train, family='binomial',alpha=1,lambda=lambda_min_lasso)
    pred_lo = predict(model_cv, newx = xB_val)
    num_val = length(yB_val)
    lr_pred = rep(0,num_val)
    actual = rep(0,num_val)
    for (j in 1:num_val){
        if (pred_lo[j]>0){
            lr_pred[j]=1
        }
        actual[j] = yB_val[j]
    }
    # Compute 0-1 loss for each observation
    lr_loss = abs(lr_pred-actual) # loss is 0 if NB_pred=actual, 1 otherwise
    # Compute mean 0-1 loss on the val set
    zoloss[i] = mean(lr_loss)
}
xB_train = xB[-(((k-1)*fsize+1):(length(yB))),]
yB_train = yB[-(((k-1)*fsize+1):(length(yB)))]
xB_val = xB[((k-1)*fsize+1):(length(yB)),]
yB_val = yB[((k-1)*fsize+1):(length(yB))]
# Fit model on training and make predictions on validation
model_cv <- glmnet(xB_train, yB_train, family='binomial',alpha=1,lambda=lambda_min_lasso)
pred_lo = predict(model_cv, newx = xB_val)
num_val = length(yB_val)
lr_pred = rep(0,num_val)
actual = rep(0,num_val)
for (j in 1:num_val){
    if (pred_lo[j]>0){
        lr_pred[j]=1
    }
    actual[j] = yB_val[j]
}
# Compute 0-1 loss for each observation
lr_loss = abs(lr_pred-actual) # loss is 0 if NB_pred=actual, 1 otherwise
# Compute mean 0-1 loss on the val set
zoloss[k] = mean(lr_loss)
test_error_est = mean(zoloss)

cat("=====================================================================\n")
cat("GLMNET Lasso Logistic Regression Model with lambda.min\n\n")
cat("Zero-One Loss (10-fold Cross-Validation Average):",test_error_est,"\n")
cat("Accuracy (10-fold Cross-Validation Average):",1-test_error_est,"\n")
cat("=====================================================================\n")

GLMNET Lasso Logistic Regression Model with lambda.min

Zero-One Loss (10-fold Cross-Validation Average): 0.06952764 
Accuracy (10-fold Cross-Validation Average): 0.9304724 


## lambda.1se

In [12]:
k = 10
n = length(train50B[,1])
fsize = round(n/k)
rmse = rep(0,k)
zoloss = rep(0,k)
for (i in 1:(k-1)){
    # Get train and validation sets
    xB_train = xB[-(((i-1)*fsize+1):(i*fsize)),]
    yB_train = yB[-(((i-1)*fsize+1):(i*fsize))]
    xB_val = xB[((i-1)*fsize+1):(i*fsize),]
    yB_val = yB[((i-1)*fsize+1):(i*fsize)]
    # Fit model on training and make predictions on validation
    model_cv <- glmnet(xB_train, yB_train, family='binomial',alpha=1,lambda=lambda_1se_lasso)
    pred_lo = predict(model_cv, newx = xB_val)
    num_val = length(yB_val)
    lr_pred = rep(0,num_val)
    actual = rep(0,num_val)
    for (j in 1:num_val){
        if (pred_lo[j]>0){
            lr_pred[j]=1
        }
        actual[j] = yB_val[j]
    }
    # Compute 0-1 loss for each observation
    lr_loss = abs(lr_pred-actual) # loss is 0 if NB_pred=actual, 1 otherwise
    # Compute mean 0-1 loss on the val set
    zoloss[i] = mean(lr_loss)
}
xB_train = xB[-(((k-1)*fsize+1):(length(yB))),]
yB_train = yB[-(((k-1)*fsize+1):(length(yB)))]
xB_val = xB[((k-1)*fsize+1):(length(yB)),]
yB_val = yB[((k-1)*fsize+1):(length(yB))]
# Fit model on training and make predictions on validation
model_cv <- glmnet(xB_train, yB_train, family='binomial',alpha=1,lambda=lambda_1se_lasso)
pred_lo = predict(model_cv, newx = xB_val)
num_val = length(yB_val)
lr_pred = rep(0,num_val)
actual = rep(0,num_val)
for (j in 1:num_val){
    if (pred_lo[j]>0){
        lr_pred[j]=1
    }
    actual[j] = yB_val[j]
}
# Compute 0-1 loss for each observation
lr_loss = abs(lr_pred-actual) # loss is 0 if NB_pred=actual, 1 otherwise
# Compute mean 0-1 loss on the val set
zoloss[k] = mean(lr_loss)
test_error_est = mean(zoloss)

cat("=====================================================================\n")
cat("GLMNET Lasso Logistic Regression Model with lambda.1se\n\n")
cat("Zero-One Loss (10-fold Cross-Validation Average):",test_error_est,"\n")
cat("Accuracy (10-fold Cross-Validation Average):",1-test_error_est,"\n")
cat("=====================================================================\n")

GLMNET Lasso Logistic Regression Model with lambda.1se

Zero-One Loss (10-fold Cross-Validation Average): 0.07403518 
Accuracy (10-fold Cross-Validation Average): 0.9259648 


## Ridge

In [30]:
# Select regularization parameter over trainA (50% of training data)
model_lasso <- cv.glmnet(xA, yA, family='binomial',alpha=0)
lambda_min_ridge = model_lasso$lambda.min
lambda_1se_ridge = model_lasso$lambda.1se
cat("lambda_min_ridge = ",lambda_min_ridge,"\n")
cat("lambda_1se_ridge = ",lambda_1se_ridge,"\n")

lambda_min_ridge =  0.02988029 
lambda_1se_ridge =  0.04335119 


## lambda.min

In [31]:
k = 10
n = length(train50B[,1])
fsize = round(n/k)
rmse = rep(0,k)
zoloss = rep(0,k)
for (i in 1:(k-1)){
    # Get train and validation sets
    xB_train = xB[-(((i-1)*fsize+1):(i*fsize)),]
    yB_train = yB[-(((i-1)*fsize+1):(i*fsize))]
    xB_val = xB[((i-1)*fsize+1):(i*fsize),]
    yB_val = yB[((i-1)*fsize+1):(i*fsize)]
    # Fit model on training and make predictions on validation
    model_cv <- glmnet(xB_train, yB_train, family='binomial',alpha=0,lambda=lambda_min_ridge)
    pred_lo = predict(model_cv, newx = xB_val)
    num_val = length(yB_val)
    lr_pred = rep(0,num_val)
    actual = rep(0,num_val)
    for (j in 1:num_val){
        if (pred_lo[j]>0){
            lr_pred[j]=1
        }
        actual[j] = yB_val[j]
    }
    # Compute 0-1 loss for each observation
    lr_loss = abs(lr_pred-actual) # loss is 0 if NB_pred=actual, 1 otherwise
    # Compute mean 0-1 loss on the val set
    zoloss[i] = mean(lr_loss)
}
xB_train = xB[-(((k-1)*fsize+1):(length(yB))),]
yB_train = yB[-(((k-1)*fsize+1):(length(yB)))]
xB_val = xB[((k-1)*fsize+1):(length(yB)),]
yB_val = yB[((k-1)*fsize+1):(length(yB))]
# Fit model on training and make predictions on validation
model_cv <- glmnet(xB_train, yB_train, family='binomial',alpha=0,lambda=lambda_min_ridge)
pred_lo = predict(model_cv, newx = xB_val)
num_val = length(yB_val)
lr_pred = rep(0,num_val)
actual = rep(0,num_val)
for (j in 1:num_val){
    if (pred_lo[j]>0){
        lr_pred[j]=1
    }
    actual[j] = yB_val[j]
}
# Compute 0-1 loss for each observation
lr_loss = abs(lr_pred-actual) # loss is 0 if NB_pred=actual, 1 otherwise
# Compute mean 0-1 loss on the val set
zoloss[k] = mean(lr_loss)
test_error_est = mean(zoloss)

cat("=====================================================================\n")
cat("GLMNET Ridge Logistic Regression Model with lambda.min\n\n")
cat("Zero-One Loss (10-fold Cross-Validation Average):",test_error_est,"\n")
cat("Accuracy (10-fold Cross-Validation Average):",1-test_error_est,"\n")
cat("=====================================================================\n")

GLMNET Ridge Logistic Regression Model with lambda.min

Zero-One Loss (10-fold Cross-Validation Average): 0.07553266 
Accuracy (10-fold Cross-Validation Average): 0.9244673 


## lambda.1se

In [15]:
k = 10
n = length(train50B[,1])
fsize = round(n/k)
rmse = rep(0,k)
zoloss = rep(0,k)
for (i in 1:(k-1)){
    # Get train and validation sets
    xB_train = xB[-(((i-1)*fsize+1):(i*fsize)),]
    yB_train = yB[-(((i-1)*fsize+1):(i*fsize))]
    xB_val = xB[((i-1)*fsize+1):(i*fsize),]
    yB_val = yB[((i-1)*fsize+1):(i*fsize)]
    # Fit model on training and make predictions on validation
    model_cv <- glmnet(xB_train, yB_train, family='binomial',alpha=0,lambda=lambda_1se_ridge)
    pred_lo = predict(model_cv, newx = xB_val)
    num_val = length(yB_val)
    lr_pred = rep(0,num_val)
    actual = rep(0,num_val)
    for (j in 1:num_val){
        if (pred_lo[j]>0){
            lr_pred[j]=1
        }
        actual[j] = yB_val[j]
    }
    # Compute 0-1 loss for each observation
    lr_loss = abs(lr_pred-actual) # loss is 0 if NB_pred=actual, 1 otherwise
    # Compute mean 0-1 loss on the val set
    zoloss[i] = mean(lr_loss)
}
xB_train = xB[-(((k-1)*fsize+1):(length(yB))),]
yB_train = yB[-(((k-1)*fsize+1):(length(yB)))]
xB_val = xB[((k-1)*fsize+1):(length(yB)),]
yB_val = yB[((k-1)*fsize+1):(length(yB))]
# Fit model on training and make predictions on validation
model_cv <- glmnet(xB_train, yB_train, family='binomial',alpha=0,lambda=lambda_1se_ridge)
pred_lo = predict(model_cv, newx = xB_val)
num_val = length(yB_val)
lr_pred = rep(0,num_val)
actual = rep(0,num_val)
for (j in 1:num_val){
    if (pred_lo[j]>0){
        lr_pred[j]=1
    }
    actual[j] = yB_val[j]
}
# Compute 0-1 loss for each observation
lr_loss = abs(lr_pred-actual) # loss is 0 if NB_pred=actual, 1 otherwise
# Compute mean 0-1 loss on the val set
zoloss[k] = mean(lr_loss)
test_error_est = mean(zoloss)

cat("=====================================================================\n")
cat("GLMNET Ridge Logistic Regression Model with lambda.1se\n\n")
cat("Zero-One Loss (10-fold Cross-Validation Average):",test_error_est,"\n")
cat("Accuracy (10-fold Cross-Validation Average):",1-test_error_est,"\n")
cat("=====================================================================\n")

GLMNET Ridge Logistic Regression Model with lambda.1se

Zero-One Loss (10-fold Cross-Validation Average): 0.07703518 
Accuracy (10-fold Cross-Validation Average): 0.9229648 


## Best Lasso Model

In [32]:
best_model <- glmnet(xB, yB, family='binomial',alpha=1,lambda=lambda_min_lasso)

In [33]:
fitted_coef <- coef(best_model)

In [34]:
fitted_coef

181 x 1 sparse Matrix of class "dgCMatrix"
                                               s0
(Intercept)                            0.43258927
animalNameG1                          -0.43788900
animalNameG2                           .         
animalNameG3                           0.60268131
animalNameG4                           1.15443858
animalNameG5                          -0.88261012
animalNameHCN1                         3.95489335
animalNameHCNb2                       -2.20361742
animalNameHCNb4                        .         
animalNameHCNd1                        .         
animalNameHCNd2                       -0.53378286
animalNameHCNe1                        .         
animalNameHCNe2                        .         
animalNameHCNe3                        .         
animalNamenpI1                         .         
totalCellNum                           0.08646754
gender                                 .         
genotype                               .         
weight_

## Bootstrap with Lasso for confidence intervals

In [37]:
df = train50B
coef.boot = function(data, indices) {
    dataT = data[indices,]
    xB = model.matrix(ketBool ~ . + animalName:correlationScore
                    + animalName:lickAccuracy
                    + animalName:lickNumber
                    + animalName:avgFR
                    + animalName:avgSingleCellVariance
                    + animalName:varianceFR
                    + animalName:avgTrialSpeed
                    + animalName:varianceSpeed-1, data = dataT)
    yB = dataT$ketBool
    fm = glmnet(xB, yB, family='binomial',alpha=1,lambda=lambda_min_lasso)
    return(coef(fm))
}

In [ ]:
boot.out5000 = boot(df, coef.boot, 5000)
boot.out5000